Moving Average Crossover Strategy

In [22]:
import numpy as np 
import pandas as pd 
import yfinance as yf 

def _stock_prices_dataset(ticker): 
  
  #import data from yahoo finance 
  stock_prices_dataset = yf.download(ticker, start="2020-01-01", end="2024-01-01") 
  
  stock_prices_dataset = stock_prices_dataset[["Adj Close"]].reset_index(drop=True) 
   
  stock_prices_dataset["returns"] = np.log(stock_prices_dataset["Adj Close"]).diff() 
  
  stock_prices_dataset["returns"] = stock_prices_dataset["returns"].shift(-1) 

  return stock_prices_dataset 

In [23]:
def _sma_stock_prices_dataset(stock_prices_dataset, long_sma = 200, short_sma = 50):

  stock_prices_dataset[f"{long_sma}_day_sma"] = stock_prices_dataset["Adj Close"].rolling(long_sma).mean()
  stock_prices_dataset[f"{short_sma}_day_sma"] = stock_prices_dataset["Adj Close"].rolling(short_sma).mean()
  stock_prices_dataset = stock_prices_dataset.dropna()
  stock_prices_dataset = stock_prices_dataset.reset_index(drop=True)

  return stock_prices_dataset

In [24]:
def _sma_crossover_dataset(sma_stock_prices_dataset, long_sma = 200, short_sma = 50):

  sma_crossover_datalist = []

  sma_stock_prices_dataset["crossover_signal"] = np.where(sma_stock_prices_dataset[f"{short_sma}_day_sma"] >= sma_stock_prices_dataset[f"{long_sma}_day_sma"], 1, 0)
  sma_stock_prices_dataset["prev_crossover_signal"] = sma_stock_prices_dataset["crossover_signal"].shift(1, fill_value=0)

  sma_stock_prices_dataset["buy"] = (sma_stock_prices_dataset["crossover_signal"] == 1) & (sma_stock_prices_dataset["prev_crossover_signal"] == 0)
  sma_stock_prices_dataset["buy"]  = sma_stock_prices_dataset["buy"] .replace({True: 1, False: 0})
  sma_stock_prices_dataset["sell"] = (sma_stock_prices_dataset["crossover_signal"] == 0) & (sma_stock_prices_dataset["prev_crossover_signal"] == 1)
  sma_stock_prices_dataset["sell"]  = sma_stock_prices_dataset["sell"] .replace({True: 1, False: 0})
  
  holding = False
  for sma_stock_prices in sma_stock_prices_dataset.to_dict("records"):
    
    if holding and sma_stock_prices.get("sell") == 1:
      holding = False
    
    if not holding and sma_stock_prices.get("buy") == 1:
      holding = True

    sma_stock_prices["hold"] = holding
    sma_crossover_datalist.append(sma_stock_prices)
  
  sma_crossover_dataset = pd.DataFrame(sma_crossover_datalist)

  return sma_crossover_dataset

In [25]:
print(_sma_crossover_dataset(_sma_stock_prices_dataset(_stock_prices_dataset("AAPL"))))

[*********************100%%**********************]  1 of 1 completed

      Adj Close   returns  200_day_sma  50_day_sma  crossover_signal  \
0    118.259949 -0.014100    86.583096  114.545799                 1   
1    116.604240 -0.025874    86.800820  114.650088                 1   
2    113.625961  0.013106    87.007204  114.745462                 1   
3    115.124893 -0.005461    87.218200  114.839171                 1   
4    114.497902 -0.009630    87.427776  114.986029                 1   
..          ...       ...          ...         ...               ...   
801  194.431885 -0.005563   177.831922  184.808136                 1   
802  193.353287 -0.002845   178.049736  185.070357                 1   
803  192.803986  0.000518   178.275199  185.358695                 1   
804  192.903839  0.002224   178.491362  185.651624                 1   
805  193.333298 -0.005439   178.699129  185.984461                 1   

     prev_crossover_signal  buy  sell  hold  
0                        0    1     0  True  
1                        1    0     0  True

In [26]:
def _sma_returns_data(sma_crossover_dataset):

  sma_crossover_dataset["algo_returns"] = sma_crossover_dataset["returns"] * sma_crossover_dataset["hold"]
  sma_return = round(sma_crossover_dataset["algo_returns"].sum() * 100, 3)
  buy_and_hold_return = round(sma_crossover_dataset["returns"].sum() * 100, 3)
  total_entries = sma_crossover_dataset["buy"].sum()

  return sma_return, buy_and_hold_return, total_entries

In [27]:
def sma_handler(ticker, long_sma = 200, short_sma = 50):

  stock_prices_dataset = _stock_prices_dataset(ticker)
  # start_date = stock_prices_dataset.iloc[0]['date']
  sma_stock_prices_dataset = _sma_stock_prices_dataset(stock_prices_dataset, long_sma, short_sma)
  sma_crossover_dataset = _sma_crossover_dataset(sma_stock_prices_dataset, long_sma, short_sma)
  sma_return, buy_and_hold_return, total_entries = _sma_returns_data(sma_crossover_dataset)
  
  # print(f"Start Date: {start_date}")
  print(f"SMA Return: {sma_return}")
  print(f"Buy and Hold Return: {buy_and_hold_return}")
  print(f"Total Entries: {total_entries}")

sma_handler("AAPL")


[*********************100%%**********************]  1 of 1 completed


SMA Return: 32.588
Buy and Hold Return: 48.609
Total Entries: 3
